# Random Environment with Three Stages Repairmen Model

This example demonstrates a random environment model with three operational stages and illustrates the computation of the infinitesimal generator.

## Model Features:
- **Environment**: 3 stages (UP, DOWN, FAST)
- **Queueing Network**: 2 stations per stage (Delay + PS Queue)
- **Job Population**: 2 jobs
- **Circulant Transitions**: Environment transitions follow a circulant pattern
- **Generator Computation**: Demonstrates infinitesimal generator matrix calculation

## Environment Stages:
1. **Stage1 (UP)**: Rate[0]=3, Rate[1]=1
2. **Stage2 (DOWN)**: Rate[0]=2, Rate[1]=2  
3. **Stage3 (FAST)**: Rate[0]=1, Rate[1]=3

The circulant transition matrix creates a cyclic dependency between stages.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.env.*
import jline.solvers.ctmc.*
import jline.util.matrix.*

In [ ]:
// Helper function to create circulant matrix
fun circulant(size: Int): Array<DoubleArray> {
    val matrix = Array(size) { DoubleArray(size) }
    // Create circulant matrix where last element of each row is 1
    for (i in 0 until size) {
        matrix[i][(i + size - 1) % size] = 1.0
    }
    return matrix
}

// Helper function to generate a queueing network for random environment
fun renvGenqn(rates: DoubleArray, N: Int): Network {
    val qnet = Network("qn1")
    
    val node1 = Delay(qnet, "Queue1")
    val node2 = Queue(qnet, "Queue2", SchedStrategy.PS)
    
    val jobclass = ClosedClass(qnet, "Class1", N, node1, 0)
    
    node1.setService(jobclass, Exp(rates[0]))
    node2.setService(jobclass, Exp(rates[1]))
    
    val P = qnet.initRoutingMatrix()
    P.set(jobclass, jobclass, node1, node2, 1.0)
    P.set(jobclass, jobclass, node2, node1, 1.0)
    qnet.link(P)
    
    return qnet
}

In [ ]:
// Model parameters
val N = 2  // Job population  
val M = 2  // Number of stations
val E = 3  // Number of environment stages

// Create environment model
val envModel = Env("MyEnv", E)
val envName = arrayOf("Stage1", "Stage2", "Stage3")
val envType = arrayOf("UP", "DOWN", "FAST")

// Create rate matrix
// rate[stage][station] = service rate for station in given stage
val rates = Array(E) { stage ->
    doubleArrayOf(
        (E - stage).toDouble(),  // First station: 3,2,1
        (stage + 1).toDouble()   // Second station: 1,2,3
    )
}

println("Rate matrix:")
rates.forEachIndexed { stage, stageRates ->
    println("Stage $stage: ${stageRates.contentToString()}")
}

In [ ]:
// Create queueing networks for each environment stage
val envSubModels = Array(E) { stage ->
    renvGenqn(rates[stage], N)
}

// Add stages to environment model
for (e in 0 until E) {
    envModel.addStage(e, envName[e], envType[e], envSubModels[e])
}

In [ ]:
// Define environment transition rates using circulant matrix
val envRates = circulant(3)  // Creates a 3x3 circulant matrix

println("\nEnvironment transition rates (circulant matrix):")
envRates.forEach { row ->
    println(row.contentToString())
}

// Add transitions with Erlang distributions
for (e in 0 until E) {
    for (h in 0 until E) {
        if (envRates[e][h] > 0) {
            val meanTime = 1.0 / envRates[e][h]
            val order = kotlin.math.max(e + h, 1)  // Erlang order based on stage indices, minimum 1
            envModel.addTransition(e, h, Erlang.fitMeanAndOrder(meanTime, order))
        }
    }
}

In [ ]:
// Create solvers for each submodel using CTMC
val solvers = Array(E) { stage ->
    CTMC(envSubModels[stage])
}

// Create environment solver
val envSolver = ENV(envModel, solvers.toList())

println("\n=== Solving Three-Stage Environment Model ===")
println("The metasolver considers an environment with $E stages and queueing networks with $M stations.")
println("This example illustrates the computation of the infinitesimal generator of the system.")

In [ ]:
// Get results
try {
    println("\n=== Generator Computation ===")
    
    // Try to get the generator (infinitesimal generator matrix)
    try {
        val generator = envSolver.generator
        println("Infinitesimal generator computation completed")
        println("Generator computed successfully")
        // Note: Full generator printing might be too verbose
    } catch (genError: Exception) {
        println("Generator computation error: ${genError.message}")
        println("Note: getGenerator() may require specific configurations")
    }
    
} catch (e: Exception) {
    println("Error during generator computation: ${e.message}")
}

In [ ]:
// Alternative: Get ensemble averages
try {
    println("\n=== Ensemble Average Results ===")
    val avgTable = envSolver.ensembleAvg
    println("Ensemble average performance metrics:")
    avgTable.print()
} catch (avgError: Exception) {
    println("\nEnsemble average error: ${avgError.message}")
    println("Note: Some environment solver features may require specific solver configurations")
}